In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

In [2]:
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

In [3]:
haiku_writer_prompt = ChatPromptTemplate.from_messages([
  ("system", "You are specialist in writing Haikus about programming language."),
  ("human", "I want to read haiku about {language}."),
])

haiku_writer_chain = haiku_writer_prompt | chat

In [7]:
haiku_curator_prompt = ChatPromptTemplate.from_messages([
  ("system", "You are Haiku teacher specialized on explain Haiku about programming language. you explain haiku."),
  ("human", "{haiku}."),
])

haiku_curator_chain = haiku_curator_prompt | chat

In [8]:
haiku_chain = {"haiku": haiku_writer_chain} | haiku_curator_chain

haiku_chain.invoke({"language": "elixir"})

Elixir's magic,
Functional spells woven strong,
Concurrency dance.This Haiku beautifully captures the essence of Elixir, a programming language known for its functional programming paradigm and powerful concurrency features. The first line sets the stage by referring to Elixir as "magic," hinting at its mystical and transformative qualities. The second line highlights the elegance and strength of Elixir's functional programming capabilities, likening them to spells being woven together. Finally, the third line paints a vivid picture of the dynamic and harmonious interplay of concurrent processes in Elixir, described as a "dance." Overall, this Haiku effectively conveys the essence of Elixir as a language that empowers developers to create robust and scalable systems through its unique features.

AIMessageChunk(content='This Haiku beautifully captures the essence of Elixir, a programming language known for its functional programming paradigm and powerful concurrency features. The first line sets the stage by referring to Elixir as "magic," hinting at its mystical and transformative qualities. The second line highlights the elegance and strength of Elixir\'s functional programming capabilities, likening them to spells being woven together. Finally, the third line paints a vivid picture of the dynamic and harmonious interplay of concurrent processes in Elixir, described as a "dance." Overall, this Haiku effectively conveys the essence of Elixir as a language that empowers developers to create robust and scalable systems through its unique features.')